In [1]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D, Dropout, Embedding, GlobalAveragePooling1D, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy

import os
import re
import shutil
import string
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print('tensorflow version is {}'.format(tf.__version__))

tensorflow version is 2.8.0


In [3]:
file='stack_overflow_16k'
url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

In [4]:
# # 뭔가 이상함, cache_subdir=file로 한 후 '' 해야 directory 구조가 원하는 대로 됨
dataset = tf.keras.utils.get_file(file, origin=url, untar=True, cache_dir='.' , cache_subdir=file)

In [5]:
# # 뭔가 이상함, cache_subdir=file로 한 후 '' 해야 directory 구조가 원하는 대로 됨
dataset = tf.keras.utils.get_file(file, origin=url, untar=True, cache_dir='.' , cache_subdir='')

In [6]:
os.listdir(dataset)

['test', 'stack_overflow_16k.tar.gz', 'README.md', 'train']

In [7]:
train_dir = os.path.join(dataset, 'train')
train_dir_list = os.listdir(train_dir)

In [8]:
train_dir_list

['python', 'java', 'csharp', 'javascript']

In [9]:
train_dir_list = [i for i in train_dir_list if '.' != i[0]]

In [10]:
train_dir_list

['python', 'java', 'csharp', 'javascript']

In [11]:
indexs = np.random.randint(0, 2000, len(train_dir_list))

for idx, kind in enumerate(train_dir_list):
    file = os.listdir(os.path.join(train_dir, kind))[indexs[idx]]
    file = os.path.join(os.path.join(train_dir, kind), file)
    with open(file) as f:
        print(f.read())
    print('=========================================================')
    

"how to iterate a file in chunks? i have a file (foo.txt) that is sorted like so (column 0 is grouped):..1  foo     bar.1  lorem   ipsum   gypsum.1  baba    loo     too.2  hello   goodbye seeya.3  kobe    magic   wilt.3  foo     sneaks  bar.3  more    stuff.3  last    line    in      file...how can i iterate the file in chunks of line.split()[0]?  i know that generators can do this but i'm not entirely sure how.  essentially, i would like to do this:..def first_column_grouping(file):.    yield some_list ## how?..with open(""foo.txt"") as file:.    for group in first_column_grouping(file): ## 3 values.        print group...expected output:..[""1 foo bar"", ""1 lorem ipsum gypsum"", ""1 baba loo too""].[""2 hello goodbye seeya""].[""3 kobe magic wilt"", 3 foo sneaks bar"", ""3 more stuff"", ""3 last line in file""]"

"can't get the method to work i have this method here, which converts an image to a byte array...public byte[] imagetocompressedbytearray(image image) throws ioexception {./

In [12]:
batch_size = 32
seed = 42

In [13]:
raw_train = tf.keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Metal device set to: Apple M1


2022-06-10 10:58:09.641943: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-10 10:58:09.642048: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:
a = next(iter(raw_train.take(1)))
print(a[0][0])
print(raw_train.class_names[a[1][0]])
print(a[1][0])

tf.Tensor(b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default cons

2022-06-10 10:58:10.548965: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [15]:
for text, label in raw_train.take(1):
    for i in range(1):
        print('=================================')
        print(raw_train.class_names[label[i]])
        print('=================================')
        print(text[i].numpy())
        print('=================================')
        print(text[i])
        print('=================================')

java
b'"blank8 why is my solution faster than the neat solution? (hackerrank chocolate feast) edit: simplified my solution..edit: removed opinion based secondary question...background: atarted learning blank a week or two ago using hackerranks problems as exercises and stackoverflow search + google as my teacher, i\'ve had some limited experience learning other languages...i did the exercise my own ""noobish learner way"" which i can\'t help but feel is a ""botched job"" when i see ""neat &amp; short"" solutions...however, when submitting both solutions one after another a couple of times i found the ""neat"" solution was quite a bit slower. ..i vaguely remember something about % operations being costly, is mine faster because of no % operations or is there more to it than just that?..exercise: https://www.hackerrank.com/challenges/chocolate-feast..neat solution from discussion:..import blank.io.*;.import blank.util.*;..public class solution {.    static int cc; .    public static void

In [16]:
val_train = tf.keras.utils.text_dataset_from_directory(
    'stack_overflow_16k/train',
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [17]:
test_dir = os.path.join(dataset + '/test')
test_dir

'./stack_overflow_16k/test'

In [18]:
test = tf.keras.utils.text_dataset_from_directory(test_dir, batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [24]:
sub_dir = os.listdir(train_dir)[1]
sample_dir = os.path.join(train_dir, sub_dir)
file = os.listdir(sample_dir)[1]

In [25]:
sample_file = os.path.join(sample_dir, file)
sample_file

'./stack_overflow_16k/train/java/1735.txt'

In [26]:
print(tf.strings.lower(sample_file))

tf.Tensor(b'./stack_overflow_16k/train/java/1735.txt', shape=(), dtype=string)


In [27]:
print(tf.strings.lower(sample_file).numpy())

b'./stack_overflow_16k/train/java/1735.txt'


In [28]:
with open(sample_file) as f:
    sample_file = f.read()
    print(sample_file)

"how to adjust the speed of objects moving over a 2d playing field? i'm writing a tower defence game and my main component is a grid divided into 40x40 fields (an object of the class field). when the game starts the sortest way to the nearest base is found for every spawn (which is a special kind of field) and saved in its field int[] path, which represents a sequence of directions which the monster must follow to reach the base...i have created a thread that has a loop which moves every monster 1px forward  and increases its field distance in each pass. every time the monster moves i also check if it hasn't stepped on the center of a field. i do it using:..if ((monster.getdistance() % field_size /*which is 40*/) == 0)...and when it returns 0 the monster changes its direction (reading the position of the next field to reach from its spawn field path)...this works when every enemy moves 1px per pass. but i want them to move with different speed. i found 2 solutions:...move every enemy i

In [44]:
lowercase = tf.strings.lower(sample_file)
print(lowercase.numpy())

b'"how to adjust the speed of objects moving over a 2d playing field? i\'m writing a tower defence game and my main component is a grid divided into 40x40 fields (an object of the class field). when the game starts the sortest way to the nearest base is found for every spawn (which is a special kind of field) and saved in its field int[] path, which represents a sequence of directions which the monster must follow to reach the base...i have created a thread that has a loop which moves every monster 1px forward  and increases its field distance in each pass. every time the monster moves i also check if it hasn\'t stepped on the center of a field. i do it using:..if ((monster.getdistance() % field_size /*which is 40*/) == 0)...and when it returns 0 the monster changes its direction (reading the position of the next field to reach from its spawn field path)...this works when every enemy moves 1px per pass. but i want them to move with different speed. i found 2 solutions:...move every ene

In [45]:
stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
print(stripped_html.numpy())

b'"how to adjust the speed of objects moving over a 2d playing field? i\'m writing a tower defence game and my main component is a grid divided into 40x40 fields (an object of the class field). when the game starts the sortest way to the nearest base is found for every spawn (which is a special kind of field) and saved in its field int[] path, which represents a sequence of directions which the monster must follow to reach the base...i have created a thread that has a loop which moves every monster 1px forward  and increases its field distance in each pass. every time the monster moves i also check if it hasn\'t stepped on the center of a field. i do it using:..if ((monster.getdistance() % field_size /*which is 40*/) == 0)...and when it returns 0 the monster changes its direction (reading the position of the next field to reach from its spawn field path)...this works when every enemy moves 1px per pass. but i want them to move with different speed. i found 2 solutions:...move every ene

In [46]:
stripped_html2 = tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')
print(stripped_html2.numpy())

b'how to adjust the speed of objects moving over a 2d playing field im writing a tower defence game and my main component is a grid divided into 40x40 fields an object of the class field when the game starts the sortest way to the nearest base is found for every spawn which is a special kind of field and saved in its field int path which represents a sequence of directions which the monster must follow to reach the basei have created a thread that has a loop which moves every monster 1px forward  and increases its field distance in each pass every time the monster moves i also check if it hasnt stepped on the center of a field i do it usingif monstergetdistance  fieldsize which is 40  0and when it returns 0 the monster changes its direction reading the position of the next field to reach from its spawn field paththis works when every enemy moves 1px per pass but i want them to move with different speed i found 2 solutionsmove every enemy in a different thread the higher its speed is th

[!"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}\~]
